# Leer datos

In [17]:
import pickle
import pandas as pd

In [18]:
from pathlib import Path
dir = Path('.')
check = dir / 'info.pkl'
if check.exists():
    info = pd.read_pickle('info.pkl')
    print('Leyendo datos de info.pkl')
else:
    print('Creando info.pkl')
    datos = pd.read_pickle("../exploratorio/dict_dataframes.pkl")
    info = {}
    info['biorreactores'] = {}
    info['centrifugas'] = {}
    info['Fases producción'] ={}
    info['Cinéticos IPC'] = {}
    info['Fases test'] = {}

    biorreactores = info['biorreactores']
    centrifugas = info['centrifugas']
    fases = info['Fases producción']
    cineticos = info['Cinéticos IPC']
    fases_test = info['Fases test']
    
    for k in datos.keys():
        print(k)
        if 'Bio' in k:
            biorreactores[k.split('/')[0].split('.')[0].split(' ')[-1]] = datos[k]
        elif 'Cen' in k:
            centrifugas[k.split('/')[0].split('.')[0].split(' ')[-1]] = datos[k]
        elif 'Fases' in k and 'test' not in k:
            fases[k.split('/')[1]] = datos[k]
        elif 'test' in k:
            fases_test[k.split('/')[1]] = datos[k]
        elif 'Cinéticos' in k:
            cineticos[k.split('/')[1]] = datos[k]
        else:
            info[k.split('/')[0].split('.')[0]] = datos[k]
            
    # Hay que corregir la forma del dataframe de fases de producción de preinóculo
            
    new_columns = ['LOTE', 'Fecha/hora inicio', 'Fecha/hora fin', 'pH línea 1',
        'pH línea 2', 'pH línea 3', 'Turbidez línea 1', 'Turbidez línea 2', 'Turbidez línea 3', 'Línea usada línea 1',
        'Línea usada línea 2', 'Línea usada línea 3']
    info['Fases producción']['Preinóculo'].columns = new_columns
    info['Fases producción']['Preinóculo'] = info['Fases producción']['Preinóculo'].loc[1:]
    
    with open('info.pkl', 'wb') as file:
        pickle.dump(info, file)
        print('Info creado')

Leyendo datos de info.pkl


# Crear input

En el siguiente código voy a crear los datos con los que entrenar el modelo, de forma que iremos iterando sobre cada uno de los lotes extrayendo toda la información de ese lote en los datasets. Las funciones auxiliares para extraer la información estan en `utils.py`. Tras extraer cada uno de los inputs, se hará una imputación de los faltantes in place por la media ya que en el exploratorio se ha visto que los valores no solían ser muy dispares, por lo que la media puede ser una buena imputación.

In [19]:
import numpy as np
def aux(x: list) -> float:
    if len(x) == 0:
        return np.nan
    else:
        return x[0]

In [20]:
from utils import limpiar_string_lote
from utils import buscar_registros_biorreactor
from utils import buscar_registros_centrifuga
from utils import buscar_registros_temperaturas
from datetime import datetime

dir = Path('.')
check = dir / 'training.pkl'

if check.exists():
    with open('training.pkl', 'rb') as f:
        training = pickle.load(f)
    print('Training leído')
else:
    print('Creando training')
    training = {}

    for _,row in info['OF 123456 v02'].iterrows():
        
        orden = row['Orden']
        lote = limpiar_string_lote(row['Lote'])
        cant_entregada = row['Cantidad entregada']
        
        # INPUT FASES DE PRODUCCIÓN
        
        prod_cultivo = info['Fases producción']['Cultivo final'][info['Fases producción']['Cultivo final']['LOTE'] == lote]
        
        if len(prod_cultivo) == 0: # se trata de un lote de test
            continue
        
        producto_1 = list(prod_cultivo['Producto 1'].values)
        
        # hay que tener en cuenta si el lote tiene padre o no para buscar los datos de preinóculo e inóculo del padre
        if prod_cultivo['Orden en el encadenado'].values == 2:
            lote_parental = prod_cultivo['LOTE parental'].values[0]
            prod_preinoculo = info['Fases producción']['Preinóculo'][info['Fases producción']['Preinóculo']['LOTE'] == lote_parental]
            prod_inoculo = info['Fases producción']['Inóculo'][info['Fases producción']['Inóculo']['LOTE'] == lote_parental]
            
        if prod_cultivo['Orden en el encadenado'].values == 3:
            lote_padre = prod_cultivo['LOTE parental'].values[0]
            if np.isnan(lote_padre): # hay un lote que a pesar de ser el 3 en el orden del encadenado no tiene lote padre
                continue
            prod_cultivo_padre = info['Fases producción']['Cultivo final'][info['Fases producción']['Cultivo final']['LOTE'] == lote_padre]
            lote_abuelo = prod_cultivo_padre['LOTE parental'].values[0]
            prod_preinoculo = info['Fases producción']['Preinóculo'][info['Fases producción']['Preinóculo']['LOTE'] == lote_abuelo]
            prod_inoculo = info['Fases producción']['Inóculo'][info['Fases producción']['Inóculo']['LOTE'] == lote_abuelo]
        
        else:
            prod_preinoculo = info['Fases producción']['Preinóculo'][info['Fases producción']['Preinóculo']['LOTE'] == lote]
            prod_inoculo = info['Fases producción']['Inóculo'][info['Fases producción']['Inóculo']['LOTE'] == lote]
        
        input_prod = [
            aux(prod_preinoculo['pH línea 1'].values), aux(prod_preinoculo['pH línea 2'].values), aux(prod_preinoculo['pH línea 3'].values), 
            aux(prod_preinoculo['Turbidez línea 1'].values), aux(prod_preinoculo['Turbidez línea 2'].values), aux(prod_preinoculo['Turbidez línea 3'].values), 
            aux(prod_preinoculo['Línea usada línea 1'].values), aux(prod_preinoculo['Línea usada línea 2'].values), aux(prod_preinoculo['Línea usada línea 3'].values), 
            aux(prod_inoculo['Volumen de cultivo'].values), aux(prod_inoculo['Turbidez inicio cultivo'].values), aux(prod_inoculo['Turbidez final culttivo'].values), 
            aux(prod_inoculo['Viabilidad final cultivo'].values), aux(prod_cultivo['Orden en el encadenado'].values), 
            aux(prod_cultivo['Volumen de inóculo utilizado'].values), aux(prod_cultivo['Turbidez inicio cultivo'].values), 
            aux(prod_cultivo['Turbidez fin cultivo'].values), aux(prod_cultivo['Viabilidad final cultivo'].values), 
            aux(prod_cultivo['Centrifugación 1 turbidez'].values), aux(prod_cultivo['Centrifugación 2 turbidez'].values), 
            aux(prod_cultivo['Producto 2'].values)
        ]
        
        # INPUT BIORREACTOR 1
        
        prod_inoculo = info['Fases producción']['Inóculo'][info['Fases producción']['Inóculo']['LOTE'] == lote]
        biorreactor_1 = prod_inoculo['ID bioreactor'].values
        
        if len(biorreactor_1) == 0:
            biorreactor_1 = 'NaN'
        else:
            biorreactor_1 = biorreactor_1[0]
        fecha_inicio, fecha_fin = prod_inoculo['Fecha/hora inicio'].map(str).values, prod_inoculo['Fecha/hora fin'].map(str).values
        if len(fecha_inicio) == 0:
            fecha_inicio = 'NaN'
        else:
            fecha_inicio = fecha_inicio[0]
        if len(fecha_fin) == 0:
            fecha_fin = 'NaN'
        else:
            fecha_fin = fecha_fin[0]
            
        input_biorreactor_1 = buscar_registros_biorreactor(info, biorreactor_1, fecha_inicio, fecha_fin, 250)
        
        # BIORREACTOR 2
        
        biorreactor_2 = prod_cultivo['ID Bioreactor'].values

        if len(biorreactor_2) == 0:
            biorreactor_2 = 'NaN'
        else:
            biorreactor_2 = biorreactor_2[0]
            
        if biorreactor_2 == 13189: # no existe ese biorreactor suponemos que es un error y que se refiere al 13169
            biorreactor_2 = 13169
        
        fecha_inicio_2, fecha_fin_2 = prod_cultivo['Fecha/hora inicio'].map(str).values, prod_cultivo['Fecha/hora fin'].map(str).values
        if len(fecha_inicio_2) == 0:
            fecha_inicio_2 = 'NaN'
        else:
            fecha_inicio_2 = fecha_inicio_2[0]
        if len(fecha_fin_2) == 0:
            fecha_fin_2 = 'NaN'
        else:
            fecha_fin_2 = fecha_fin_2[0]
            
        input_biorreactor_2 = buscar_registros_biorreactor(info, biorreactor_2, fecha_inicio_2, fecha_fin_2, 250)

        # INPUT CENTRIFUGADORAS
        
        centrifugadora = prod_cultivo['ID Centrífuga'].values

        if len(centrifugadora) == 0:
            centrifugadora = 'NaN'
        else:
            centrifugadora = centrifugadora[0]
            
        horas_centri = info['Horas inicio fin centrífugas'][info['Horas inicio fin centrífugas']['Orden'] == orden]
        
        centri_1_ini = horas_centri[horas_centri['Operación'] == 'Centrifugació 1 ini']
        equipo_1 = centri_1_ini['EQUIPO'].values
        if len(equipo_1) == 0:
            equipo_1 = 'NaN'
        else:
            equipo_1 = equipo_1[0]  
        date_1_ini = centri_1_ini['DATEVALUE'].values
        if len(date_1_ini) == 0:
            date_1_ini = 'NaN'
        else:
            date_1_ini = date_1_ini[0]
            
        centri_1_fin = horas_centri[horas_centri['Operación'] == 'Centrifugació 1 fi']
        date_1_fin = centri_1_fin['DATEVALUE'].values
        if len(date_1_fin) == 0:
            date_1_fin = 'NaN'
        else:
            date_1_fin = date_1_fin[0]
            
        centri_2_ini = horas_centri[horas_centri['Operación'] == 'Centrifugació 2 ini']
        equipo_2 = centri_2_ini['EQUIPO'].values
        if len(equipo_2) == 0:
            equipo_2 = 'NaN'
        else:
            equipo_2 = equipo_2[0]  
        
        date_2_ini = centri_2_ini['DATEVALUE'].values
        if len(date_2_ini) == 0:
            date_2_ini = 'NaN'
        else:
            date_2_ini = date_2_ini[0]
            
        centri_2_fin = horas_centri[horas_centri['Operación'] == 'Centrifugació 2 fi']
        date_2_fin = centri_2_fin['DATEVALUE'].values
        if len(date_2_fin) == 0:
            date_2_fin = 'NaN'
        else:
            date_2_fin = date_2_fin[0]

        input_centrifugadoras = [buscar_registros_centrifuga(info, equipo_1, date_1_ini, date_1_fin, 100), buscar_registros_centrifuga(info, equipo_2, date_2_ini, date_2_fin, 100)]
        
        
        # INPUT TEMPERATURAS
        
        # tomamos como fecha inicial para obtener los datos de temperatura la de inicio de preinóculo y la final la última de centrifugación
        
        primera_fecha = prod_preinoculo['Fecha/hora inicio'].values
        if len(primera_fecha) == 0:
            primera_fecha = 'NaN'
        else:
            primera_fecha = primera_fecha[0]
        
        ultima_fecha = date_2_fin
        
        if ultima_fecha == 'NaN':
            input_temp = [None]
        else:
            ultima_fecha_parsed = datetime.strptime(ultima_fecha, '%m-%d-%Y %H:%M:%S')
            ultima_fecha_formated = datetime.strftime(ultima_fecha_parsed, '%Y-%m-%d %H:%M:%S')
            ultima_fecha = str(ultima_fecha_formated)
            primera_fecha = str(primera_fecha)
            input_temp = [buscar_registros_temperaturas(info, primera_fecha, ultima_fecha_formated, 400)]
        
        training[lote] = {
            'Prod' : input_prod,
            'Biorreactores' : [input_biorreactor_1, input_biorreactor_2],
            'Centrifugadora': input_centrifugadoras,
            'Temperaturas' : input_temp
        }
        
    with open('training.pkl', 'wb') as f:
        pickle.dump(training, f) 

Creando training


In [22]:
training[23019]

{'Prod': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  32.800000000000004,
  134400000,
  1,
  82.4,
  17.28,
  91.2,
  184000000,
  nan,
  nan,
  6.0],
 'Biorreactores': [None,
  [[80.0,
    0.0,
    0.6539367675781249,
    72.46521606445313,
    0.0,
    3.999715861839512,
    1636.8000000000002,
    5.881179428100584,
    -0.2347625732421872,
    0.0,
    11.879998779296882,
    0.0,
    679.2171875,
    71.9620815516952,
    5.93600006103516,
    30.064001464843763],
   [80.0,
    0.0,
    0.6539866712876968,
    64.63638916015624,
    0.0,
    4.000014507909384,
    1645.6,
    5.881471252441408,
    -0.2347625732421872,
    0.0,
    11.879998779296882,
    0.0,
    679.2171875,
    64.57806396484376,
    5.927999877929689,
    29.95200195312504],
   [80.0,
    0.0,
    0.6741882324218752,
    57.09110107421872,
    0.0,
    4.000490327582169,
    1645.6,
    5.873307037353513,
    -0.2347625732421872,
    0.0,
    11.879998779296882,
    7.34127953